In [ ]:
from TVLSI26.datsets.binary_mnist import BinaryMNIST
from TVLSI26.configs.config import modelConstants, noiseRangeConfig
from TVLSI26.neuron_models.digLIF import digLIF, Square
from TVLSI26.ctt_weights.weight_variations import maskW, WeightDropoutLinear, apply_quant_noise, add_retention_noise
from TVLSI26.models.populationEncodedBinaryClassifier import populationEncodedBinaryClassifier, _prediction_check, _population_code
from TVLSI26.ctt_weights.noisePool import NoisePool, get_noise_schedule

In [ ]:
import torch, torch.nn as nn
import torch.nn.functional as F
import snntorch as snn
from snntorch import functional as SF
from snntorch import spikegen
import numpy as np
from torch.utils.data import TensorDataset, DataLoader
import random
import time
import matplotlib.pyplot as plt
import copy
import csv
import pandas as pd

In [ ]:
seed = int(time.time() * 1000) ^ random.getrandbits(32)
counter_global = 0
results_path = "C:/Users/rezva/Documents/hardwareAwareLearning/results/"
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
dtype = torch.float

In [ ]:
batch_size = 128
index0 = 0
index1 = 8

# or mnist_dataset = augmentedDataset(index0, index1, device, augmentation_type = "HOG", FULL_MNIST = False)
mnist_dataset = BinaryMNIST(data_path=modelConstants.data_path, index0=index0, index1=index1, image_size=modelConstants.image_size)
train_loader = DataLoader(mnist_dataset.get_train_data(), batch_size=batch_size, shuffle=True, drop_last=True)
test_loader = DataLoader(mnist_dataset.get_test_data(), batch_size=batch_size, shuffle=False, drop_last=True)

data = iter(train_loader)
data_it, targets_it = next(data)

for dataset_name, dataset in zip(["mnist_train", "mnist_test"], (mnist_dataset.get_train_data(), mnist_dataset.get_test_data())):
    print(f"Number of images in {dataset_name}: {len(dataset)}")

In [ ]:
# noise pool for training
noisePool = NoisePool()
Ideal_Ids = noisePool.get_Ideal_Ids()

pool_of_noise_dist = {
    key: [val - key for val in values]
    for key, values in noisePool.pool_of_weight_dist.items()
}

In [ ]:
class Net(nn.Module):
    def __init__(self, num_inputs=modelConstants.num_inputs):
        super().__init__()
        self.num_inputs = num_inputs

        self.fc1 = WeightDropoutLinear(self.num_inputs, modelConstants.num_outputs, bias=False)
        self.lif1 = digLIF(beta=1, reset_mechanism="zero")
        self.initialize_weights()

    def initialize_weights(
        self,
    ):
        nn.init.normal_(
            self.fc1.linear.weight,
            mean=(modelConstants.max_weight - modelConstants.min_weight) / 2,
            std=(modelConstants.max_weight - modelConstants.min_weight) / 16,
        )
        with torch.no_grad():
            self.fc1.linear.weight.clamp_(modelConstants.min_weight, modelConstants.max_weight)

    def forward(self, x, noise_type="uniform", tail_frac = 0.1):
        step = 0
        mem1 = self.lif1.init_leaky()

        base_weights = self.fc1.linear.weight.clone()
        quantized_weights = maskW(base_weights)  # clamp + quantize

        noisy_weights_flat = []
        for w in quantized_weights.view(-1):
            closest_level = min(
                pool_of_noise_dist.keys(), key=lambda l: abs(l - w.item())
            )

            if noise_type == "mixed":
                mode = "tail_high" if closest_level < noisePool.ideal_Ids[6] else "tail_low"
            else:
                mode = noise_type

            noise_sample = noisePool.sample_from_pool(
                pool_of_noise_dist[closest_level],
                mode = mode,
                tail_frac=tail_frac
            )
                
            noisy_weights_flat.append(w + noise_sample) #w + noise_sample
            #print(len(noisy_weights_flat), noise_sample.dtype, w.dtype)

        noisy_weights = torch.stack(noisy_weights_flat).reshape_as(quantized_weights)

        if self.training and modelConstants.training_noise_ctt:
            if modelConstants.retention_noise:
                noisy_weights = add_retention_noise(
                    self.fc1.linear.weight, modelConstants.std_ret_high, modelConstants.std_ret_low
                )
            if modelConstants.quantization_noise:
                noisy_weights = apply_quant_noise(noisy_weights)
        else:
            noisy_weights = base_weights

        spk_rec = []
        mem_rec = []

        for step in range(x.shape[0]):
            spk0 = x[step].flatten(1)
            cur1 = Square.apply(x[step].flatten(1), noisy_weights)

            noise1 = torch.randn_like(self.fc1.linear.weight) * modelConstants.std_ctt

            if self.training and modelConstants.general_noise:
                cur1 = cur1 + nn.functional.linear(spk0, noise1)
            spk1, mem1 = self.lif1(cur1, mem1)

            spk_rec.append(spk1)
            mem_rec.append(mem1)

        return torch.stack(spk_rec, dim=0), torch.stack(mem_rec, dim=0)

In [ ]:
torch.manual_seed(seed)
net = populationEncodedBinaryClassifier().to(device)
net.fc1.linear.weight = nn.Parameter(torch.rand(net.fc1.linear.weight.shape).to(device) * modelConstants.max_weight)
net.initialize_lif_thresholds(device)

optimizer = torch.optim.Adam(net.parameters(), lr=37e-4, betas=(0.9, 0.999))
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=.5)
mse = nn.MSELoss()
mseMem = nn.MSELoss()
bce = nn.BCEWithLogitsLoss()
mseCount = SF.mse_count_loss(population_code=True, num_classes=modelConstants.num_classes)

In [ ]:
seed = int(time.time() * 1000) ^ random.getrandbits(32)
torch.manual_seed(seed)
counter = 0
step = 0
lambda_std = 0.04
lambda_min = 0.04
commenting_on = False
num_warmup_epochs = 4
num_samples = 5

def train_and_evaluate(net = net, fr_th = 0.5, loss_fn = mse, num_epochs = 5, population_code=True, optimizer = optimizer, 
                       scheduler=scheduler, train_loader = train_loader, test_loader = test_loader):

    if population_code:
        loss_fn = mseCount

    best_weights = None
    lowest_loss = 100.0
    num_epochs += num_warmup_epochs

    for epoch in range(num_epochs):
        train_batch = iter(train_loader)
        epoch_time = []
        total_loss = 0
        batch_idx = 0
        lowest_b_acc = 1.0

        start_time = time.time()
        for data, targets in train_batch:
            data = data.to(device)
            data = spikegen.latency(data, num_steps=modelConstants.num_steps, threshold=0.01, clip=True, first_spike_time=0, linear=True, normalize=True).cumsum(0)
            targets = targets.to(device)

            outputs = []
            accuracies = []
            loss_accum = 0

            noise_type, tail_frac = get_noise_schedule(epoch)

            if epoch < num_warmup_epochs:
                training_noise_ctt = False
            else:
                training_noise_ctt = True
                print("Training noise applied: ", noise_type, ", ", tail_frac)

            for i in range(num_samples):
                net.train()
                spk_rec, mem_rec = net(data, noise_type=noise_type, tail_frac=tail_frac)

                loss_sa = torch.zeros((1), dtype=dtype, device=device)
                repeated_targets = (targets.float().unsqueeze(0).repeat(modelConstants.num_steps, 1).unsqueeze(-1))
                fr = spk_rec.sum(0) / spk_rec.shape[0]

                if(loss_fn == mse):
                    fr = spk_rec.sum(0) / spk_rec.shape[0]
                    loss_sa = loss_fn(fr.squeeze(), targets.float()) #mse
                elif(loss_fn == bce):   
                    loss_tensor = loss_fn(mem_rec - modelConstants.Threshold_voltage, repeated_targets)
                    loss_sa = loss_tensor.sum()
                elif(loss_fn == mseMem):
                    repeated_targets *= modelConstants.Vdd
                    loss_tensor = loss_fn(mem_rec, repeated_targets)
                    loss_sa = loss_tensor.sum()
                elif(loss_fn == mseCount):  
                    loss_sa = mseCount._compute_loss(spk_rec, targets)

                loss_accum += loss_sa

                if population_code:
                    _, _, num_outputs = _prediction_check(spk_rec)
                    _, predicted_ = _population_code(spk_rec, modelConstants.num_classes, num_outputs).max(1)
                else:
                    fr = spk_rec.sum(0) / spk_rec.shape[0]  # Compute firing rate
                    predicted_ = (fr.squeeze() > fr_th).int()

                outputs.append(fr)
                acc = (predicted_ == targets).sum().item() / targets.size(0)
                if acc < lowest_b_acc:
                    lowest_b_acc = acc
                accuracies.append(acc)

            optimizer.zero_grad()
            avg_acc = sum(accuracies) / len(accuracies)
            stacked_outputs = torch.stack(outputs)

            avg_pred = stacked_outputs.mean(dim=0) + 1e-8
            safe_outputs = stacked_outputs + 1e-8
            kl_div = (safe_outputs * (safe_outputs / avg_pred).log()).mean()

            min_acc = min(accuracies)
            min_acc_penalty = lambda_min * (1.0 - min_acc)

            loss_val = (
                (loss_accum / num_samples) + (lambda_std * kl_div) + min_acc_penalty
            )
            loss_val.backward()
            #gradients = net.fc1.linear.weight.grad
            optimizer.step()

            total_loss += loss_val.item()
            end_time = time.time()
            batch_idx += 1
            with torch.no_grad():
                stacked_out = torch.stack(outputs)
                std_across_samples = stacked_out.std(dim=0).mean().item()
                if commenting_on:
                    print(
                        f"Epoch {epoch+1} | Batch {batch_idx} | Loss: {loss_val.item():.4f} | Output std: {std_across_samples:.4f}"
                    )
                print(
                    f"[E{epoch+1} B{batch_idx}] Avg acc: {avg_acc:.2f} | Min acc: {min(accuracies):.2f} | Max acc: {max(accuracies):.2f}"
                )

            with torch.no_grad():
                net.fc1.linear.weight.clamp_(0, modelConstants.max_weight)

        if commenting_on:
            print(f"epoch time {end_time - start_time:.2f}s")
            print("epoch loss: ", loss_val)

        epoch_time.append(end_time - start_time)
        scheduler.step()

        with torch.no_grad():
            net.eval()
            total = 0
            correct = 0

            for data, targets in test_loader:
                data = data.to(device)
                data = spikegen.latency(data, num_steps=modelConstants.num_steps, threshold=0.01, clip=True, first_spike_time=0, linear=True, normalize=True,).cumsum(0)
                targets = targets.to(device)
                spk_rec, _ = net(data)

                if population_code:
                    correct += SF.accuracy_rate(spk_rec, targets, population_code=population_code, num_classes=num_classes, ) * spk_rec.size(1)
                else:
                    fr = spk_rec.sum(0) / spk_rec.shape[0]
                    predicted = (fr.squeeze() > fr_th).int()
                    correct += (predicted == targets).sum().item()

                total += targets.size(0)

            if commenting_on:
                print(f"Total correctly classified test set images: {correct}/{total}")
                print(f"Test Set Accuracy: {100 * correct / total:.2f}%\n")
            for param_group in optimizer.param_groups:
                current_lr = param_group["lr"]
                if commenting_on:
                    print(f"Current learning rate: {current_lr}")
        if (total_loss / len(train_loader)) < lowest_loss:
            lowest_loss = total_loss / len(train_loader)
            best_weights = net.fc1.linear.weight.clone()
        print(
            f"\nEpoch {epoch+1} Summary: Avg Loss = {total_loss / len(train_loader):.4f}\n"
        )
    print("average epoch time: ", np.mean(epoch_time))
    net.fc1.linear.weight.data.copy_(best_weights)
    return 100 * (correct / total)

In [ ]:
def evaluate_binary_classifier(net=net, test_loader=test_loader, fr_th=0.4):
    with torch.no_grad():
        net.eval()
        total = 0
        correct = 0
        for data, targets in test_loader:
            data = data.to(device)
            data = spikegen.latency(data, num_steps=num_steps, threshold=0.01, clip=True, first_spike_time=0, linear=True, normalize=True).cumsum(0)
            targets = targets.to(device)
            spk_rec, _ = net(data)

            if modelConstants.population_code:
                correct += SF.accuracy_rate(spk_rec, targets, population_code=modelConstants.population_code, num_classes=modelConstants.num_classes) * spk_rec.size(1)
            else:
                fr = spk_rec.sum(0) / spk_rec.shape[0]
                predicted = (fr.squeeze() > fr_th).int()
                correct += (predicted == targets).sum().item()

        total += targets.size(0)

        print(f"Total correctly classified test set images with noisy weights: {correct}/{total}")
        print(f"Test Set Accuracy with noisy weights: {100 * correct / total:.2f}%\n")
        return 100 * correct / total

In [ ]:
from itertools import product
best_accuracy = 0
best_params = None
worst_accuracy = 100 

thr_values = [0.3]
fr_th_values = [0.5]

hyperparams = [{"thr": thr, "fr_th": fr_th} for thr, fr_th in product(thr_values, fr_th_values)]
optimizer = torch.optim.Adam(net.parameters(), lr=1e-3, betas=(0.9, 0.999))
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=2)

for params in hyperparams:
    fr_th = params["fr_th"]
    V_th = params["thr"]
    net.initialize_lif_thresholds(device, V_th=V_th)
    accuracy = train_and_evaluate(fr_th=fr_th, loss_fn=mseMem, num_epochs = 20, population_code=modelConstants.population_code, optimizer = optimizer, scheduler=scheduler, train_loader = train_loader, test_loader = test_loader)
    print(f"Accuracy with params {params}: {accuracy}")

    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_params = params
        best_weights = net.fc1.linear.weight.clone()
    if accuracy < worst_accuracy:
        worst_accuracy = accuracy
        worst_params = params
        
    print(f"Best parameters: {best_params} with accuracy: {best_accuracy}")

In [ ]:
min_samples = 80 # minimum samples to start convergence check
max_samples = 200 # to prevent infinite loop
threshold = 0.001  # acceptable relative change (0.1%)
accuracies = []

previous_mean = None
previous_std = None
sample_idx = 0

while True:
    sample_idx += 1

    for i in range(4):
        model = copy.deepcopy(net)
        perturbed_state_dict = model.state_dict()

        for name, param in perturbed_state_dict.items():
            if 'weight' in name:
                with torch.no_grad():
                    perturbed_weights = []
                    for w in param.view(-1).cpu().numpy():
                        closest_level = min(Ideal_Ids, key=lambda x: abs(x - w))
                        if i==0:
                            noisy_value = np.random.choice(noisePool.pool_of_weight_dist[closest_level])
                        elif i==1:
                            noisy_value = noisePool.sample_from_pool(noisePool.pool_of_weight_dist[closest_level], mode="uniform", original_weight=w)
                        elif i==2:
                            noisy_value = noisePool.sample_from_pool(noisePool.pool_of_weight_dist[closest_level], mode="uniform", original_weight=w)
                        elif i==3:
                            if closest_level < Ideal_Ids[6]:
                                noisy_value = noisePool.sample_from_pool(noisePool.pool_of_weight_dist[closest_level], mode="tail_high", tail_frac=0.001, p_static=0.0, original_weight=w)
                            else:
                                noisy_value = noisePool.sample_from_pool(noisePool.pool_of_weight_dist[closest_level], mode="tail_low", tail_frac=0.001, p_static=0.0, original_weight=w)
                        perturbed_weights.append(noisy_value)
                    param.copy_(torch.tensor(np.array(perturbed_weights).reshape(param.shape), dtype=param.dtype))

        model.load_state_dict(perturbed_state_dict)
        acc = evaluate_binary_classifier(net=model)
        accuracies.append(acc)
        print(f"Sample {sample_idx}: Accuracy = {acc:.2f}%, Mean = {np.mean(accuracies):.2f}%, Std = {np.std(accuracies):.2f}%")
    
    if sample_idx >= min_samples:
        current_mean = np.mean(accuracies)
        current_std = np.std(accuracies)

        if previous_mean is not None and previous_std is not None:
            mean_change = abs(current_mean - previous_mean) / previous_mean
            std_change = abs(current_std - previous_std) / previous_std

            if mean_change < threshold and std_change < threshold:
                print(f"Converged after {sample_idx} samples.")
                break

        previous_mean = current_mean
        previous_std = current_std

    if sample_idx >= max_samples:
        print(f"Reached maximum samples ({max_samples}) without full convergence.")
        break

print(f"Mean accuracy: {np.mean(accuracies):.2f}%")
print(f"Best-case accuracy: {np.max(accuracies):.2f}%")
print(f"Worst-case accuracy: {np.min(accuracies):.2f}%")
print(f"Std: {np.std(accuracies):.2f}%")

plt.hist(accuracies, bins=10)
plt.xlabel('Accuracy')
plt.ylabel('Frequency')
plt.title('Monte Carlo Sampling Accuracy Distribution')
plt.show()

In [ ]:
print("minimum = ", np.min(accuracies))
print("maximum = ", np.max(accuracies))
print("mean = ", np.mean(accuracies))
print("std = ", np.std(accuracies))

### Generating a CSV file with the quantized CTT weights

In [ ]:
# based on hardware simulations and CTT-supported weight levels 
weights_L = [0.020000, 0.05327272727272728, 0.08654545454545455, 0.11981818181818182, 
             0.15309090909090908, 0.18636363636363637, 0.21963636363636363, 0.2529090909090909, 
             0.2861818181818182, 0.3194545454545455, 0.3527272727272728, 0.386000]
threshold_values = [110, 75, 55, 51, 43, 35, 23, 18, 14, 10, 7, 0]
def quantize_value(w):
    if (w>=modelConstants.max_weight):
      return modelConstants.max_weight
    elif (w<=modelConstants.min_weight):
      return modelConstants.min_weight
    else:
      step = (modelConstants.max_weight - modelConstants.min_weight) / (modelConstants.num_w_levels - 1)
      return round((w - modelConstants.min_weight) / step) * step + modelConstants.min_weight
weights_np = []
weights_np.append(0)
replacement_index = 0
first_neuron = False
if first_neuron:
   max_index = 100
   min_index = 0
else:
   max_index = 200
   min_index = 100

In [ ]:
trained_weights = best_weights
if(modelConstants.pretrained_weights_used):
    for i in range(min_index, min(max_index, trained_weights.view(-1).numel())):
        if first_neuron:
            j = i + 1
        else:
            j= i + 1 - 100
        weight = trained_weights.view(-1)[i].item()
        weight = quantize_value(weight)
        replacement_index = weights_L.index(weight)
        print(f"V{j}={threshold_values[replacement_index]}m ",end='')
        weights_np.append(threshold_values[replacement_index]) 

In [ ]:
if(not modelConstants.pretrained_weights_used):
    for name, param in net.named_parameters():
            if 'weight' in name:
                for i in range(min_index, min(max_index, param.numel())):
                    if first_neuron:
                         j = i + 1
                    else:
                        j= i + 1 - 100
                    weight = param.view(-1)[i].item()
                    weight = quantize_value(weight)
                    replacement_index = weights_L.index(weight)
                    print(f"V{j}={threshold_values[replacement_index]}m ",end='')
                    weights_np.append(threshold_values[replacement_index])

### Generating OCEAN script for Cadence input

In [ ]:
def hardwareImage(pix):
    modifiedPixel = 336 - pix*20
    return modifiedPixel

def softwareImage(modifiedPixel):
    pix = (336 - modifiedPixel) / 20
    return pix

num_data_points = len(mnist_dataset.get_test_data())
test_loader_h = DataLoader(mnist_dataset.get_test_data(), batch_size=num_data_points, shuffle=True, drop_last=True)

In [ ]:
fr_np = []

for data, targets in test_loader_h:
    data = data.to(device)
    data_preSNN = data
    data = spikegen.latency(data, num_steps=modelConstants.num_steps, threshold=0.01, clip=True, first_spike_time=0, linear=True, normalize=True).cumsum(0)

    if(modelConstants.pretrained_weights_used):
        with torch.no_grad():  # Disable gradient tracking
            net.fc1.linear.weight.data.copy_(trained_weights)

    spk_rec, mem = net(data)
    #fr = (spk_rec.sum(0) / spk_rec.shape[0])
    fr = spk_rec.sum(0)
    summed_data = torch.sum(data, axis=0)
print(data.shape)
print(summed_data.shape, num_data_points)

hardwareInput = hardwareImage(summed_data)

inputData = np.zeros((num_data_points, modelConstants.num_inputs))

for k in range(num_data_points):
    temp_image = hardwareInput[k]
    flat_image = temp_image.view(-1)

    for pixel in range (flat_image.numel()):
        inputData[k][pixel] = flat_image[pixel]

test_data = inputData.T

filename_inData = f'{results_path}/inputData{num_data_points}39ideal_v{counter_global}.csv'
with open(filename_inData,'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerows(test_data)

for elem in fr.view(-1):
    fr_np.append(elem.detach().cpu().item())

firing_rate = np.array(fr_np).reshape(-1, 1)

filename = f'{results_path}/firing_rate{num_data_points}39ideal_v{counter_global}.csv'
with open(filename,'w') as h:
    csv.writer(h).writerows(firing_rate)
counter_global += 1

In [ ]:
torch.set_printoptions(edgeitems=torch.numel(fr), threshold=torch.numel(fr))
fr[0:num_data_points, 1]

In [ ]:
from datetime import datetime
current_datetime = datetime.now()
datetime_string = current_datetime.strftime("%Y_%m_%d_%H_%M_%S")
hardware_filename = "bc_" + datetime_string
hardware_filename

In [ ]:
# generating text file for a batch of test inputs
filename = f'{results_path}/sim_{num_data_points}_v{counter_global}.txt'
fout = open(filename, "wt")
fout.write("ii = 0 \n")

for k in range(num_data_points):
    fout.write('''simulator( 'spectre )
design( "DigNeuron_New" "EntireNeuronArchV2SingleCell_Rezvan" "schematic")
resultsDir( "/data/shared/brainer/home/zmoham12/simulation/DigNeuron_New/EntireNeuronArchV2SingleCell_Rezvan/maestro/results/maestro/.tmpADEDir_zahra.mohammadrezaee/DigNeuron_New_EntireNeuronArchV2SingleCell_1/simulation/EntireNeuronArchV2SingleCell_Rezvan/spectre/schematic" )
modelFile( 
    '("$SPECTRE_MODEL_PATH/design_wrapper.lib.scs" "tt_pre")
)
analysis('tran ?stop "340n"  )
desVar(	  "VG" 110m	)
desVar(	  "VD" 0.8	)
desVar(	  "f" 20n	)
''')
    temp_image = hardwareInput[k]
    flat_image = temp_image.view(-1)

    for pixel in range (flat_image.numel()):
        A = 'desVar(       "Vd{}" {}n )'
        fout.write(A.format(pixel,flat_image[pixel]) + "\n")

    fout.write(f'''
desVar(	  "wx" 0m	)
envOption(
	'analysisOrder  list("pz" "dcmatch" "stb" "tran" "envlp" "ac" "dc" "lf" "noise" "xf" "sp" "pss" "pac" "pstb" "pnoise" "pxf" "psp" "qpss" "qpac" "qpnoise" "qpxf" "qpsp" "hb" "hbac" "hbstb" "hbnoise" "hbxf" "sens" "acmatch")
	'switchViewList  list("spectre" "spectreText" "cmos_sch" "cmos.sch" "schematic" "veriloga")
)
option(	'pzSeverity  "None"
	'noiseSeverity  "None"
	'spSeverity  "None"
	'acSeverity  "None"
	'dcOpSeverity  "None"
	'dcSeverity  "None"
	'tranSeverity  "None"
)
temp( 27 )
out = outfile("results{hardware_filename}.txt" "a")
run()
selectResult( 'tran )
outputs()
ocnPrint(?output out sample( VT("/Fire") 0 340n "linear" 1n) )
close(out)
for( i 1 30  printf( "%d \\n" ii ))
ii = ii+1
''')
fout.close()

In [ ]:
test_data = np.genfromtxt('C:/Users/rezva/Documents/hardwareAwareLearning/results/inputData2019_v2.csv', delimiter=',', dtype=np.float32)
inputData = test_data.T
total_elements = inputData.size
num_data_points = total_elements // (modelConstants.image_size * modelConstants.image_size)

if total_elements % (modelConstants.image_size * modelConstants.image_size) != 0:
    raise ValueError("Input data cannot be reshaped into the desired dimensions. Check the CSV file format.")
hardwareInput_reconstructed = torch.tensor(inputData).view(num_data_points, 1, modelConstants.image_size * modelConstants.image_size)
print("Reconstructed tensor shape:", hardwareInput_reconstructed.shape)


### Reading Cadence Results

In [ ]:
# Define the input text file path and output CSV file path
input_file = r'C:/Users/rezva/Documents/hardwareAwareLearning/results/resultsideal_39_2025_01_13_20_54_22_.txt'  # Replace with your text file path
output_file = r'C:/Users/rezva/Documents/hardwareAwareLearning/results/CadenceResults.csv'

# Read the text file and save it as a CSV
# Adjust delimiter as needed based on the text file's structure (default: space-separated)
try:
    # Read the text file into a DataFrame
    df = pd.read_csv(input_file, delimiter=r'\s+', engine="python")
    
    # Save the DataFrame as a CSV file
    df.to_csv(output_file, index=False)
    print(f"File has been successfully converted and saved as {output_file}")
except Exception as e:
    print(f"An error occurred: {e}")

In [ ]:
data0 = pd.read_csv("C:/Users/rezva/Documents/hardwareAwareLearning/results/CadenceResults.csv")
data1 = np.asarray(data0.iloc[:, 1])
num_data_points = 40

In [ ]:
y_test_predicted_sim = np.zeros([num_data_points,])

stay = True
rising_edge = True
rise_voltage = 0.35
fall_voltage = 0.3
data_index = 0

csv_file_path = "C:/Users/rezva/Documents/hardwareAwareLearning/results/processed_cadence.csv"
if(modelConstants.generate_processed_data):
  with open(csv_file_path, mode="w", newline="") as file:
      writer = csv.writer(file)
      writer.writerow(["(s)"])

for prediction_index in range(num_data_points):
  while stay:
    # Extracting values one by one from excell
    temp = str(data1[data_index])
    prefix = temp[-1]

    # Checking if we have a new input
    if temp == '(s)':
      #print("done")
      rising_edge = True
      if(modelConstants.generate_processed_data):
        with open(csv_file_path, mode="a", newline="") as file:
          writer = csv.writer(file)
          writer.writerow(["(s)"]) 
      data_index += 1
      break

    # Checking to see if there are any prefixes and replacing them with the proper value
    if prefix == 'm':
      temp = temp[:-1]
      temp = float(temp)
      temp *= 1e-3
    elif prefix == 'u':
      temp = temp[:-1]
      temp = float(temp)
      temp *= 1e-6
    elif prefix == 'nano':
      temp = temp[:-1]
      temp = float(temp)
      temp *= 1e-9
    elif prefix == 'p':
      temp = temp[:-1]
      temp = float(temp)
      temp *= 1e-12
    else :
      temp = 0

    if(modelConstants.generate_processed_data):
      with open(csv_file_path, mode="a", newline="") as file:
        writer = csv.writer(file)
        writer.writerow([temp]) 

    # Checking if there was a spike
    if (temp > rise_voltage) and not rising_edge:
      rising_edge = True
      y_test_predicted_sim[prediction_index] += 1

    if (temp < fall_voltage) and rising_edge:
      rising_edge = False
      
    if data_index < (data1.size - 1):
      data_index += 1
    else:
      break

In [ ]:
print("max value: ", y_test_predicted_sim.max(), "min value: ", y_test_predicted_sim.min())
np.set_printoptions(edgeitems=np.size(y_test_predicted_sim), threshold=np.size(y_test_predicted_sim))
y_test_predicted_sim

In [ ]:
fr_software_results = np.array([ 9.,  8.,  9., 11., 11.,  6.,  8.,  9., 11., 11.,  6.,  6.,  4.,  8.,
         9.,  5.,  8.,  5., 10.,  7.,  8.,  9.,  6., 12., 12., 11., 12.,  6.,
         5.,  8.,  9., 10., 10., 10., 11.,  5.,  8.,  8.,  6.,  6.,  9., 10.,
         8.,  8.,  9.,  7.,  8.,  8.,  7.,  4., 13., 10.,  4.,  5.,  5.,  9.,
         9., 10., 12.,  5.,  9., 10., 11., 11., 11.,  9.,  6., 13.,  6., 11.,
         9.,  7., 12., 14.,  6., 10., 10., 12.,  6.,  6., 12.,  9., 11., 12.,
        11.,  9.,  9., 10.,  8.,  7., 11., 12.,  4.,  6., 10.,  9., 11.,  5.,
         6.,  3.,  8.,  7.,  6., 10.,  9.,  8., 13., 11., 13., 12., 15.,  4.,
        11.,  4., 10., 12., 10.,  5.,  5.,  8.,  8.,  6., 10.,  9.,  8., 11.,
         5.,  6.,  7., 10.,  6., 11.,  5.,  9.,  9., 10., 12.,  9.,  8.,  7.,
         9.,  7.,  7.,  6.,  9.,  9., 11.,  4., 12.,  9., 13.,  8.,  4., 11.,
         8., 11.,  5., 12.,  9.,  3.,  6., 12.,  9., 10.,  7.,  9.,  8.,  8.,
        12.,  7., 11., 12.,  7.,  9.,  8.,  9.,  5., 10., 10.,  9.,  6.,  3.,
         3.,  8., 10.,  4.,  3., 10., 11.,  5., 12., 13.,  9.,  4.,  8.,  7.,
         6.,  6.,  8.,  8.,  6., 11.,  8., 11.,  5.,  6., 10.,  6.,  5.,  8.,
        12.,  9.,  5.,  9.,  9., 11.,  3., 11.,  4.,  7.,  7., 14.,  7.,  4.,
         9.,  7.,  7.,  8.,  3., 10., 12.,  6., 10., 11.,  9., 10.,  5.,  9.,
         4.,  8.,  6.,  5.,  6., 10.,  6., 10., 13.,  7.,  8.,  5.,  4.,  7.,
         3.,  8.,  7.,  5., 10., 12.,  6.,  6., 12.,  8.,  6.,  5.,  7.,  9.,
         9., 13.,  3.,  5.,  9.,  4.,  6.,  8., 12.,  4.,  9.,  9.,  8., 11.,
        11.,  6., 13.,  6.,  8.,  4., 12.,  8., 11.,  4.,  6., 10.,  6., 10.,
        10.,  7.,  9.,  6., 11.,  9.,  6.,  4.,  9., 10.,  6., 13., 11., 11.,
         8.,  7.,  5.,  5.,  5.,  4.,  5.,  5., 12.,  8.,  6., 10.,  6., 11.,
         7.,  7., 11.,  4., 10.,  3.,  7., 10.,  7., 10., 10.,  7., 11.,  8.,
         9., 11., 14., 10.,  6.,  4., 10.,  3.,  9., 11., 11.,  6.,  8.,  8.,
        10., 10.,  5.,  8.,  9., 11., 11.,  8.,  3., 10., 11.,  8., 10., 10.,
         8., 11.,  5.,  7.,  4., 10.,  4.,  8., 10.,  7.,  7.,  6., 10.,  6.,
         8.,  9.,  8.,  5.,  8.,  4., 13.,  3., 12.,  4., 10., 11.,  5., 10.,
         9., 12.,  5.,  9., 12.,  9.,  6., 11.,  9., 10.,  6.,  4.,  6.,  9.,
         6.,  5., 10., 11.,  9., 11.,  6.,  5.,  8., 12., 11.,  3.,  6.,  9.,
        11.,  7., 12., 10.,  8.,  7.,  6.,  3.,  5.,  6.,  9., 10.,  9.,  9.,
         6.,  8.,  6.,  4., 15., 11., 12., 11.,  6., 11.,  6., 10.,  8., 14.,
         6.,  4.,  7.,  8., 10.,  7., 10., 11.,  9.,  9.,  4.,  6.,  9.,  7.,
         8.,  6.,  6.,  8.,  5.,  9.,  7., 11.,  9.,  7.,  5., 13., 10.,  4.,
         7.,  9.,  7.,  9.,  5.,  4., 12.,  7., 10., 11.,  8., 11., 10.,  7.,
         7.,  5.,  6., 10., 10.,  5.,  5.,  9.,  4.,  4.,  8.,  9.,  7.,  6.,
         5.,  9.,  6.,  9.,  9., 10.,  9.,  3.,  6.,  8., 11.,  9.,  9.,  9.,
         7., 12.,  4., 14.,  4., 14.,  6.,  7., 12., 12.,  6.,  8.,  9.,  6.,
        11.,  9.,  4.,  8.,  7.,  8.,  4.,  8.,  9., 11.,  3.,  8.,  5.,  5.,
        12.,  9., 10., 11.,  4., 10.,  9.,  8.,  7.,  6., 11.,  5.,  7.,  6.,
        10., 13., 14.,  9., 11.,  6., 10., 10.,  6.,  8., 12.,  6., 11., 10.,
        13.,  9.,  3.,  7.,  9., 10., 10.,  9.,  3., 10., 10., 11., 11.,  5.,
        11., 10., 10., 10., 10.,  6.,  7.,  8., 11.,  3.,  9.,  9.,  8.,  4.,
        10.,  9., 12., 12.,  9.,  8., 10.,  9., 10.,  5.,  9., 12.,  7.,  6.,
        12., 14.,  7., 10., 10.,  4., 11.,  6.,  6.,  7., 10., 11.,  9.,  9.,
        12.,  9.,  9.,  7., 12.,  8., 12.,  7., 10.,  8.,  9., 10.,  6., 11.,
         9., 15.,  6.,  6.,  9., 11., 11.,  4.,  8., 12., 14., 11.,  5.,  9.,
        12.,  7.,  7.,  5., 13., 11., 10., 12.,  7., 12., 11.,  7., 10.,  7.,
         9.,  6.,  4., 15., 10.,  7., 12., 12., 12.,  6.,  4.,  9.,  5.,  4.,
         7.,  7.,  4.,  8.,  5.,  8.,  5.,  5.,  5.,  5., 11.,  5., 10.,  5.,
        10.,  7.,  6., 10.,  5.,  7., 11.,  7.,  7.,  5.,  4., 12.,  9., 12.,
         7., 11.,  9., 10.,  8.,  6.,  7.,  9.,  5., 10., 10., 10.,  5.,  9.,
         5.,  9.,  9.,  4., 11., 10.,  9.,  6., 10.,  7.,  6.,  7.,  6.,  6.,
         8., 11., 10., 11.,  9.,  8.,  5., 10., 12.,  6.,  7., 12.,  7.,  7.,
        12.,  8.,  7.,  8.,  8., 11.,  5.,  8., 10., 11., 12.,  9.,  7., 11.,
        11., 10.,  6.,  5., 12.,  8.,  6.,  8., 10.,  7.,  8., 10.,  8., 10.,
         8.,  5.,  8.,  9.,  8.,  8.,  8.,  8., 10.,  7.,  8., 12.,  5., 10.,
        12.,  4.,  4.,  7., 15.,  6., 12., 12.,  9.,  5.,  6.,  9.,  6.,  4.,
         5., 11.,  9.,  5.,  4.,  4.,  7., 11.,  7.,  6.,  6.,  8.,  6.,  9.,
        10.,  9.,  5.,  9.,  6., 13., 10.,  5.,  6.,  9., 13.,  9., 11.,  4.,
         8.,  8., 10.,  5., 12., 11., 11., 11.,  9.,  6., 12., 10.,  8.,  7.,
        13.,  6., 12., 11.,  8., 14.,  8., 11.,  7.,  7.,  5.,  7.,  8.,  7.,
         7.,  9.,  9.,  4.,  5., 13., 14., 14., 11., 12.,  9.,  8., 11., 12.,
         8.,  6.,  9.,  5.,  9.,  9., 11., 11.,  4., 10., 11.,  8., 12.,  7.,
         5.,  4.,  8., 10.,  9.,  6.,  9.,  7., 11.,  5.,  7., 15.,  5.,  9.,
         5., 13.,  8., 11.,  6., 11.,  5.,  8.,  6.,  5.,  8.,  7.,  8.,  8.,
        15.,  8., 13.,  9., 13., 10.,  4., 10.,  5., 10.,  8.,  6., 10.,  2.,
         7.,  6., 15., 10.,  5.,  8., 12.,  8., 11.,  6.,  8., 12.,  4.,  5.,
         3., 10., 13.,  6.,  5., 11., 10.,  4., 12.,  5.,  9.,  4., 11.,  6.,
         9.,  5.,  9., 14., 13., 10.,  9.,  9., 11., 11., 10.,  8.,  8.,  7.,
        10., 12.,  7.,  4., 11., 10., 12.,  7.,  8.,  6.,  7., 13.,  5.,  8.,
         6.,  5.,  6., 12., 11.,  6.,  8.,  9.,  8., 10.,  9., 11.,  8.,  4.,
        10., 12.,  8.,  6.,  6.,  7., 10.,  9.,  7., 11., 11., 11.,  6., 12.,
         8.,  8.,  8.,  9.,  8., 12., 11.,  6.,  8.,  4.,  4., 11.,  6., 10.,
        10.,  7., 12.,  9.,  5., 11., 14.,  6.,  5.,  9., 10., 14.,  6.,  5.,
        10.,  7.,  7.,  5., 13.,  6., 10., 12., 10.,  8.,  8.,  7.,  8., 13.,
         7.,  6., 11.,  9.,  9., 13.,  7.,  7.,  8.,  5.,  7.,  4.,  9.,  7.,
        11., 10., 13., 12., 12., 12.,  4.,  9., 10.,  6., 13.,  9., 12., 10.,
        10., 13., 12.,  7.,  8.,  7.,  7., 10.,  7.,  9., 11.,  7.,  5.,  5.,
         9.,  6.,  5.,  8.,  6.,  5.,  9.,  3.,  8.,  6.,  5.,  9., 11.,  7.,
        11.,  8.,  8., 11., 11., 13.,  9., 10.,  6.,  8.,  6.,  8.,  6.,  7.,
        14., 10.,  9.,  6.,  7., 10.,  5.,  8., 13.,  9.,  4.,  5., 10.,  5.,
        11.,  7., 12.,  8.,  6.,  4., 11.,  5.,  5., 10., 13.,  7.,  7.,  6.,
         4.,  9., 12.,  8.,  9.,  8.,  8.,  8., 14.,  9., 11., 10., 13.,  7.,
         9.,  6., 11.,  8., 12., 11.,  6.,  6.,  7., 11., 10., 11.,  9.,  5.,
         4.,  4., 11.,  9., 10., 13., 11.,  5., 12.,  4.,  7., 10.,  5.,  6.,
         9., 11.,  6., 11.,  8.,  7.,  9.,  8.,  6., 11.,  8., 10., 12.,  7.,
         6., 10.,  9., 10.,  7.,  7.,  7., 10., 12.,  8., 10.,  8.,  8., 12.,
         6.,  8., 12., 11., 11., 12., 10.,  5.,  5.,  4., 11., 10.,  8., 13.,
        10., 10.,  9.,  3.,  7.,  9., 10., 10.,  6.,  8.,  5., 11.,  6.,  6.,
         4.,  9.,  3.,  3.,  9.,  8.,  8.,  9.,  5.,  5.,  5.,  8.,  5.,  6.,
        11.,  8., 10., 10.,  7.,  7.,  4.,  5., 10.,  6.,  8.,  9.,  5.,  4.,
         3., 10.,  6., 11.,  4.,  6.,  7.,  4., 10.,  7.,  6.,  9., 15., 11.,
         8., 10.,  8.,  8.,  7., 11.,  9.,  9., 11.,  7., 12., 10., 12.,  8.,
         5.,  9.,  5., 10., 11., 12.,  9., 10.,  7., 11.,  5.,  5., 13., 15.,
        10., 11.,  5.,  6.,  4.,  8.,  4., 11.,  6., 11., 14., 11., 11.,  9.,
         9.,  7.,  7.,  7., 11.,  8., 11.,  6.,  9., 11.,  9., 11., 11.,  7.,
        10.,  7.,  9., 12.,  6.,  4.,  5.,  4.,  9.,  4., 10., 10.,  4., 12.,
        12., 12., 11.,  5.,  9.,  3., 10., 10.,  5.,  9.,  7., 13.,  5., 13.,
        11.,  7.,  6.,  9., 10.,  3.,  9.,  7., 13.,  8.,  4., 11.,  7.,  4.,
        13., 10., 10., 11.,  9.,  5.,  5., 10.,  6., 11.,  3.,  7.,  4., 12.,
         7.,  9., 10., 11., 10., 13., 10.,  5.,  5., 13.,  7.,  5.,  5.,  6.,
         8.,  6.,  8.,  9.,  9.,  4.,  4.,  7.,  4.,  8., 11.,  7.,  9., 10.,
         5.,  8.,  8., 12., 11.,  5.,  4., 10., 10.,  8., 12., 11., 12.,  8.,
        11., 13., 12.,  7.,  6., 12.,  7.,  4.,  5.,  7., 10., 10., 13.,  8.,
         9.,  8., 12.,  4., 10., 11.,  8.,  6.,  8., 10., 10.,  9., 10.,  6.,
         6., 10.,  7., 12.,  6.,  5.,  4., 14.,  5.,  8., 14.,  7.,  7.,  5.,
         9., 11.,  5.,  9.,  5.,  4.,  6.,  5.,  9.,  7.,  9., 12.,  5.,  7.,
         6., 10.,  5., 10.,  4.,  8., 13.,  7., 11., 11., 12.,  5., 11.,  9.,
        10.,  6.,  6.,  5.,  6.,  5., 10.,  9., 10., 10., 11.,  7.,  4.,  5.,
         6., 13., 10.,  5.,  5., 13.,  5.,  6.,  7.,  5.,  9.,  5.,  7., 10.,
         7.,  8., 10.,  8., 12., 10., 10., 12.,  7.,  8., 10., 10.,  6.,  7.,
        12.,  5.,  7.,  9.,  6.,  7.,  6., 10.,  8., 13., 10., 10., 13.,  8.,
         8.,  5.,  9.,  8., 11., 10., 12., 14.,  4., 10.,  6.,  5., 10.,  8.,
        12.,  9.,  9., 10., 12.,  9., 11.,  8.,  9.,  4.,  5.,  8.,  6., 10.,
        11., 11.,  6.,  9.,  7., 10.,  6.,  9.,  6.,  8.,  8., 11.,  9.,  7.,
        10., 10.,  4., 11.,  4.,  7.,  7.,  9., 12.,  6.,  4., 12.,  9., 11.,
         6., 10.,  7.,  5., 10., 14.,  8.,  6., 12., 11.,  9.,  7.,  9.,  8.,
        11.,  7.,  7.,  7.,  8., 11., 10.,  5., 10., 14.,  5., 10.,  5., 10.,
         8.,  5.,  7.,  7., 11., 11., 11., 10., 11.,  6., 12.,  4.,  8.,  4.,
        11.,  8.,  8.,  7., 11.,  7.,  8.,  8.,  8.,  9.,  8., 10., 11., 11.,
        10., 10., 12.,  5.,  8., 11.,  4.,  6.,  4., 10., 11., 12.,  6., 11.,
         8.,  9.,  4.,  9.,  8.,  8.,  6.,  6.,  8.,  2., 14.,  7.,  5.,  8.,
         8.,  8.,  7., 10., 11.,  5.,  6.,  9., 11.,  9., 12.,  7.,  7.,  4.,
         8.,  6.,  5.,  5.,  6.,  9.,  7., 11.,  8., 11., 10., 11.,  4., 10.,
        11.,  8.,  7.,  4., 11., 12.,  5., 11., 11.,  6.,  5.,  9.,  4.,  8.,
         9.,  6., 12., 12.,  9.,  8.,  6.,  5.,  8., 12.,  8., 12.,  5., 11.,
        12., 12.,  5.,  4.,  5.,  9.,  9., 10.,  8., 10.,  8.,  9.,  8., 12.,
         8.,  6.,  7., 10., 11.,  9., 13.,  6.,  5., 13., 11.,  9.,  9.,  7.,
         7.,  7.,  7.,  9.,  4., 10., 12.,  9.,  6., 10.,  9.,  8.,  9., 11.,
        12.,  7.,  8.,  7., 11., 10.,  4., 11.,  9., 11., 11., 10., 11., 11.,
         6., 10.,  7., 10., 12., 11., 11.,  4.,  7.,  7., 13.,  2.,  4., 11.,
         4.,  9.,  9.,  8.,  9.,  9.,  4., 10.,  8., 11.,  8.,  6.,  9.,  8.,
         6., 14., 10.,  8., 10., 11.,  7.,  6.,  5.,  8.,  4.,  7., 12.,  9.,
        11.,  7., 10.,  5.,  8.,  3.,  5.,  7.,  4., 10.,  7.,  9.,  8., 10.,
         8.,  9.,  3.,  8., 10.,  9.,  9.,  9.,  7., 10.,  7.,  7., 10.,  8.,
         9.,  9., 12.,  3.,  9.,  9.,  7., 11., 13.,  9., 11., 10.,  4.,  8.,
        10., 10., 10.,  5.,  7.,  6., 10., 10.,  4.,  4.,  8., 11.,  3., 11.,
         8.,  8., 10.,  9.,  5.,  3., 12., 10.,  7.,  6.,  7.,  8.,  9., 11.,
         6., 11.,  8., 10., 12.,  7.,  9.,  7.,  9.,  6., 13., 11.,  6., 13.,
         7.,  7.,  9., 14., 10.,  8.,  7., 11., 11., 11.,  6.,  9.,  7.,  9.,
         9.,  7., 11.,  5.,  6.,  8.,  9.,  7.,  8.,  8.,  8., 10., 10.,  8.,
         6., 12.,  8.,  9.,  7.,  9.,  9.,  8.,  2.,  4.,  6.,  7., 13., 13.,
         7.,  4.,  7.,  9., 11.,  4.,  8.,  9.,  9., 11.,  9.,  4.,  8., 11.,
         4.,  7.,  9.])

In [ ]:
print("max value: ", fr_software_results.max(), "min value: ", fr_software_results.min())

In [ ]:
# Calculate the difference between corresponding elements
difference = fr_software_results - y_test_predicted_sim

# Calculate the error percentage
error_percentage = np.abs(difference)

# Print the error percentage array
print("max err: ", error_percentage.max(), "min err: ", error_percentage.min(), "average err", np.mean(error_percentage))
print("Error percentage:", error_percentage)

In [ ]:
# Accuracy considering both neurons
if modelConstants.num_outputs % modelConstants.num_classes != 0:
    raise ValueError(
        f"`num_outputs` ({modelConstants.num_outputs}) must be a multiple of `num_classes` ({modelConstants.num_classes})."
    )

fr_0 = torch.tensor([11.,  9.,  7.,  7.,  7.,  8.,  7., 12.,  5.,  8.,  7.,  9.,  7.,
        5., 12.,  8.,  6.,  7.,  9.,  8.,  3.,  9., 11.,  5.,  7.,  6.,
        5., 10.,  9.,  3.,  4.,  5.,  5.,  5.,  5., 10., 11.,  5.,  8.,
        7.,  4.,  4.,  5.,  4., 10., 10.,  9., 10., 10.,  8.,  7.,  8.,
        7.,  9., 11., 13.,  5.,  6.,  5.,  6.,  5.,  5.,  4.,  6.,  9.,
       10., 12., 12.,  3.,  4.,  5.,  5., 10.,  7., 10.,  5.,  9.,  7.,
        9., 12.,  9.,  9., 13.,  7.,  6.,  5.,  5., 11.,  9., 12.,  6.,
        8.,  7.,  5.,  5.,  5., 11., 10.,  8.,  7., 11.,  4., 10.,  5.,
       10., 11.,  6.,  5.,  6.,  6., 11.,  8.,  6.,  8.,  5.,  5.,  9.,
        3., 10.,  4., 11.,  7.,  5.,  4.,  4.,  5.,  4.,  9., 13., 12.,
       12.,  5., 10.,  6.,  5.,  5.,  8.,  6.,  6.,  4.,  3.,  3.,  5.,
       11., 10.,  9.,  5.,  7.,  4.,  4.,  5.,  9.,  9., 12.,  4., 11.,
        9.,  8.,  5.,  7.,  7.,  6.,  4.,  5., 10., 10.,  4.,  8.,  7.,
        4.,  6.,  6., 11.,  8.,  3.,  4., 11.,  4.,  5., 10.,  5.,  8.,
        7., 10.,  5.,  7.,  7.,  5.,  4.,  7.,  4.,  6.,  6.,  9.,  4.,
        8.,  7., 10., 12.,  5., 11.,  4.,  4.,  8., 10.,  9., 10.,  8.,
        8., 11.,  5.,  9., 11., 10., 12., 10.,  6.,  4.,  9., 11., 13.,
        9., 12.,  8.,  6.,  4., 10.,  9.,  7.,  6.,  6.,  6., 11.,  5.,
        4.,  5.,  8., 11.,  8.,  4.,  4.,  6.,  7.,  4., 10.,  6., 12.,
        4.,  6., 11.,  8.,  9.,  7.,  4., 12.,  9.,  4.,  5.,  9., 11.,
        6.,  4.,  9.,  7., 11.,  9.,  5., 13.,  7.,  7., 11.,  5., 13.,
       11.,  4., 10.,  4., 13.,  3.,  4., 11., 12.,  8.,  9., 12.,  9.,
        7.,  4.,  4.,  8.,  9.,  5., 12.,  5.,  4., 10.,  4., 12.,  5.,
        6., 10.,  8.,  5.,  5., 12.,  8.,  6.,  6.,  4.,  4.,  7.,  8.,
       12.,  4.,  8., 10.,  6.,  4., 10.,  6., 11.,  5.,  5., 10.,  9.,
        9.,  6.,  2.,  4.,  5., 10., 13.,  6., 12.,  7.,  4.,  4.,  5.,
        8.,  5.,  8., 11., 11.,  7.,  6.,  5.,  7., 10.,  8.,  4.,  6.,
        5.,  6., 10., 12.,  5.,  6.,  4.,  4., 10.,  6.,  4.,  4.,  5.,
        4.,  5., 11.,  9.,  9.,  5.,  9.,  5.,  6.,  8.,  8.,  9.,  5.,
        4., 11.,  4.,  4.,  8.,  5.,  7.,  5.,  6.,  6.,  9.,  5.,  6.,
        9.,  8.,  3.,  6., 10.,  7.,  8., 10.,  8.,  6.,  5., 11., 11.,
        5.,  8.,  5.,  9.,  5.,  9.,  5.,  4.,  5.,  7.,  5.,  4.,  5.,
        4.,  4.,  9.,  4.,  4.,  9.,  5.,  4.,  9., 10., 12.,  5.,  7.,
        3., 10.,  5.,  4.,  9.,  2., 11.,  7.,  8., 12.,  6.,  6.,  5.,
       11.,  6.,  8.,  5., 10., 12., 11.,  7., 10.,  4.,  5.,  4.,  8.,
        3.,  5.,  5.,  6.,  3.,  4., 12., 13.,  7.,  8., 11.,  8.,  4.,
        9.,  9., 10.,  9.,  7.,  6.,  6.,  6.,  5.,  6., 10., 10., 10.,
        8.,  6., 10.,  5., 12., 12., 13.,  4., 11.,  4., 11.,  9.,  5.,
        4., 10.,  3.,  5.,  8.,  7., 11.,  9., 10., 10.,  9.,  3.,  9.,
        7.,  7., 10.,  7.,  5.,  7., 11.,  7.,  4.,  5., 11., 10.,  7.,
        8., 14.,  8.,  6.,  3.,  4.,  6.,  7., 12.,  9.,  4.,  9.,  4.,
        5.,  8.,  4., 12., 11.,  8.,  8.,  8.,  6.,  7.,  5.,  4.,  8.,
        5.,  5., 13.,  5.,  8.,  5., 13., 12.,  5., 13.,  4., 11.,  8.,
       12.,  4.,  8.,  7., 14.,  6.,  7.,  5.,  6.,  8.,  8.,  6., 10.,
        5.,  6.,  7.,  3.,  6., 11.,  6.,  7.,  5.,  4.,  6.,  5.,  5.,
       12.,  7.,  6.,  5.,  5.,  4., 12.,  6., 12., 11.,  9., 12.,  5.,
       11.,  9.,  4., 10.,  6., 11.,  9., 13., 12.,  4.,  6., 11., 10.,
       12., 10.,  6., 10., 11.,  6.,  7.,  4.,  9.,  5.,  3.,  4., 11.,
        9., 11.,  5.,  7.,  9.,  3., 10.,  4., 10.,  3.,  6., 11.,  9.,
       11.,  9.,  4.,  5.,  5.,  9.,  5.,  5., 12.,  9., 10.,  5.,  8.,
        5.,  4., 10.,  6., 11.,  6.,  9.,  4.,  8.,  7.,  4., 10.,  5.,
        8.,  5.,  9., 10.,  6.,  4., 11., 10.,  9., 12.,  3.,  9.,  7.,
       11., 10.,  8.,  5.,  6.,  7.,  9., 14., 12.,  7.,  9.,  3., 10.,
       11., 11.,  6., 10., 10.,  8., 11.,  6.,  7.,  5., 10.,  5.,  9.,
        9., 12.,  9., 10.,  6., 11.,  9.,  9.,  8., 12.,  4.,  6.,  4.,
        5.,  4.,  6.,  4.,  9.,  9., 10.,  9.,  5.,  4., 12.,  9.,  5.,
        8.,  5.,  4.,  5.,  4., 10., 10.,  8., 12.,  5.,  7.,  9.,  8.,
        7.,  9.,  7., 11.,  6.,  5., 11.,  7.,  8.,  5.,  7.,  8.,  9.,
        8., 11.,  9.,  5.,  5.,  5.,  6.,  5., 12., 11.,  5., 11., 11.,
       12., 12.,  6., 11.,  4.,  9., 12.,  6., 10.,  5.,  4., 10.,  6.,
        5.,  5.,  5.,  6.,  5.,  9.,  8.,  6.,  5., 11., 11.,  4., 11.,
       11.,  7.,  6., 10.,  5.,  6.,  5., 10., 13.,  8., 12.,  7.,  7.,
        4.,  6., 11.,  4.,  8.,  6., 12.,  6.,  6., 10.,  9., 10.,  3.,
       10., 11., 13., 10., 11.,  7.,  4.,  5.,  5.,  8.,  5.,  4.,  6.,
        5.,  7., 11.,  5.,  7.,  4.,  6.,  9.,  3.,  4.,  5.,  9.,  7.,
        9.,  5.,  5.,  4.,  3.,  5.,  8.,  3.,  7.,  7.,  3.,  6.,  5.,
        4.,  7.,  4.,  9., 10.,  3.,  7., 11.,  4.,  4.,  4.,  5.,  8.,
        9., 10.,  6.,  6.,  6.,  4.,  7., 12., 14.,  4.,  6.,  4.,  8.,
        4.,  9.,  7., 11.,  5.,  6.,  9.,  7., 12., 11.,  5., 11.,  8.,
        8.,  4.,  4., 10.,  8.,  6.,  6.,  5.,  7.,  7.,  8.,  8., 11.,
        6.,  7., 10.,  7.,  4.,  5.,  6.,  7.,  8.,  9.,  6., 10.,  5.,
       11.,  9., 13.,  9.,  4.,  8., 10., 10.,  4.,  7., 13.,  3.,  9.,
        7.,  6., 12.,  4.,  8.,  6., 12., 13.,  8.,  5.,  6.,  7.,  4.,
        6.,  8., 10.,  9., 13.,  5.,  9.,  8.,  5.,  9.,  9.,  6.,  9.,
       11.,  7.,  5., 10., 12., 10.,  4.,  7.,  6.,  4.,  4., 12.,  6.,
        5.,  6., 12., 11.,  9.,  8.,  7.,  9.,  9.,  5.,  5.,  8.,  4.,
       10., 10., 10., 12.,  8., 10.,  8., 11.,  5.,  6.,  6.,  8.,  4.,
        6.,  5.,  5.,  9.,  5., 14., 11.,  6.,  5.,  9., 12.,  9., 10.,
       11.,  4.,  6.,  5.,  6.,  4., 11.,  6.,  4.,  4., 11.,  4., 11.,
        6.,  5., 10.,  9.,  6., 10.,  6., 11.,  8., 11., 10.,  8., 11.,
        9.,  9.,  9.,  8., 12.,  7., 12., 12.,  7.,  8.,  4.,  7.,  9.,
        8.,  9.,  9., 10.,  6.,  5.,  5., 12., 10.,  4.,  7., 10.,  8.,
        4.,  4.,  9., 11., 12.,  9.,  4., 10., 13.,  9.,  8.,  4.,  4.,
       13.,  6.,  5.,  6.,  6.,  8., 13., 10.,  9.,  9.,  6.,  5.,  4.,
        4.,  7., 12.,  4.,  3., 14., 11., 11., 11.,  4., 11., 10., 10.,
       10., 10.,  7., 10.,  4.,  3., 12.,  5.,  7., 10., 11.,  8.,  5.,
       11., 10.,  3.,  4., 11.,  7., 14.,  6., 11., 10.,  9., 11., 10.,
        8., 10., 10.,  6.,  4.,  4., 12., 11.,  4.,  9.,  3.,  7.,  5.,
        7.,  4.,  4.,  9.,  6.,  9.,  5.,  4.,  4.,  9.,  6., 10.,  9.,
        6.,  6.,  5., 12.,  9.,  9.,  4.,  6.,  3.,  5.,  4.,  4.,  8.,
        7.,  4.,  5.,  5.,  8.,  4.,  8., 10., 15.,  5.,  6.,  6., 10.,
        5.,  6., 12.,  5., 11.,  4.,  8.,  7.,  9.,  7.,  4.,  5.,  5.,
        5., 10.,  5.,  9.,  8.,  5., 10., 11.,  5.,  5., 13.,  5., 12.,
        9.,  5.,  5., 12.,  6.,  7., 13., 10.,  7., 10.,  6.,  7.,  5.,
        8., 12.,  8., 10.,  7., 10., 12., 12.,  7.,  6.,  9.,  9.,  6.,
        5.,  5.,  6.,  4., 10.,  8., 10.,  5.,  7., 14.,  6.,  6.,  5.,
        8.,  7.,  4., 10.,  5.,  5.,  2.,  7., 11.,  6.,  8., 11.,  5.,
        5.,  9.,  6.,  5.,  4.,  3.,  5., 10.,  9.,  8.,  5.,  8., 11.,
        5.,  4.,  8.,  4.,  8., 12.,  8., 11.,  6., 10., 10.,  4.,  5.,
        8.,  6.,  4.,  9.,  4.,  8., 10., 10., 10., 15., 11., 10.,  9.,
       12.,  7.,  5., 12., 12.,  7., 13.,  5., 11., 12.,  7.,  4.,  5.,
        4.,  6.,  9., 11.,  4.,  9.,  4.,  6.,  6.,  9.,  5.,  6.,  6.,
        8.,  8.,  7.,  8.,  5.,  4.,  8.,  9., 10.,  4.,  7.,  6., 10.,
        6.,  7., 13.,  4.,  5.,  8.,  4.,  7.,  7.,  6.,  4.,  5.,  8.,
        8.,  6., 13.,  6.,  4.,  3.,  5.,  9.,  5.,  9., 11.,  9.,  6.,
        9.,  4.,  8.,  5.,  8.,  8., 14.,  6.,  4.,  6.,  9.,  6.,  5.,
       13.,  5., 10., 10.,  3., 12., 10.,  6.,  5.,  2.,  8.,  5.,  5.,
        7., 12.,  3., 14., 11.,  9.,  6., 14.,  9.,  6., 13.,  5.,  6.,
        5., 10., 10.,  9., 10.,  6.,  8.,  3., 10.,  6.,  9.,  9.,  5.,
        5.,  6.,  6.,  8.,  7.,  6.,  4.,  4., 11.,  4.,  4., 10.,  4.,
        4.,  8.,  5.,  9., 10., 10.,  8.,  6.,  5.,  8., 10.,  4.,  7.,
       14.,  4.,  8., 10.,  7., 10.,  4.,  8., 14.,  4.,  9.,  6.,  8.,
        5.,  7.,  9.,  9., 12.,  6., 11.,  8.,  7.,  9.,  6.,  4.,  5.,
        9.,  9., 13.,  7.,  9.,  8., 11., 11.,  4.,  9.,  6.,  5., 11.,
        6., 13., 10.,  5.,  4.,  5.,  7.,  5.,  7.,  8.,  8.,  3.,  9.,
        4.,  9.,  9.,  4.,  4.,  5.,  5.,  9.,  9.,  8.,  9.,  5.,  8.,
        7.,  5., 12.,  8., 10., 10.,  9.,  5.,  8., 11.,  9.,  9.,  5.,
        4.,  6.,  8.,  7.,  4.,  4., 10., 10., 11.,  8.,  7.,  9.,  6.,
        4.,  5.,  8.,  9.,  9.,  8., 10.,  5., 11., 10.,  8.,  7.,  7.,
       10.,  7.,  9., 10.,  8.,  4.,  5.,  8.,  9., 12., 13.,  4.,  4.,
       12.,  9.,  3., 11.,  6.,  4.,  5.,  9., 13.,  5.,  3.,  6., 10.,
        8.,  7.,  4., 14.,  6.,  5.,  5.,  6.,  7., 10.,  8.,  5., 11.,
        7.,  5.,  8.,  6.,  9.,  7.,  9., 10.,  8., 10.,  5.,  5.,  4.,
        6.,  7., 10.,  5.,  8., 11.,  9.,  6.,  5.,  4.,  6.,  7.,  7.,
       11., 11.,  5.,  3.,  8.,  4.,  9.,  4.,  9.,  6.,  4., 10.,  4.,
        5., 10.,  4.,  9., 12.,  9.,  6.,  9.,  8.,  7.,  6.,  5.,  8.,
        9.,  5., 11., 11., 12.,  8.,  5., 12., 10., 11.,  4.,  8., 11.,
        4.,  5.,  5.,  8.,  6.,  4.,  7.,  5.,  9.,  5., 12.,  7.,  5.,
        5.,  5., 12.,  9.,  8.,  9.,  4.,  5.,  5.,  7.,  5.,  6.,  6.,
        7.,  6., 10.,  5.,  6.,  4.,  5.,  7.,  8.,  9., 10.,  4.,  5.,
       13.,  5.,  5., 11., 13., 11.,  9.,  8., 11.,  5., 11.,  9.,  7.,
        7.,  5.,  4., 11.,  5.,  4.,  4.,  7.,  4.,  5.,  4.,  9.,  7.,
        4.,  7., 10.,  4., 12., 11.,  8.,  6.,  3., 10., 12.,  7.,  7.,
        5.,  6.,  5.,  4., 11.,  7.,  9.,  4., 12., 10.,  7., 11., 12.,
        8., 11., 10., 13., 12.,  5.,  7.,  6.,  4., 11.,  4.,  8., 10.,
        5.,  8.,  6.,  5.,  4.,  7.,  4.,  3., 11.,  9.,  9.,  7.,  5.,
        5.,  5.,  6., 12., 10.,  5., 10., 12.,  7.,  7.,  7.,  6.,  9.,
        6., 11.,  9.,  5.,  7.,  4.,  8.,  6.,  7., 12.,  6.,  4., 11.,
       13.,  7.,  6.,  4.,  8.,  6.,  9.,  6.,  9.,  4.,  5., 14.,  8.,
        3., 13., 12.,  2.,  5., 14.,  5.,  8.,  7.,  6., 10.,  4.,  7.,
        6.,  7., 11.,  4.,  7.,  5.,  6.,  6.,  3., 10.,  5.,  5.,  6.,
        5., 10.,  9., 14.,  5., 13.,  6.,  6.,  7.,  6., 10.,  7.,  4.,
        9.,  6.,  9.,  4., 12., 10.,  6.,  4.,  8.,  5.,  4.,  4.,  4.,
        9., 12.,  4.,  4.,  6.,  4.,  9.,  4.,  6.,  9.,  8.,  7.,  5.,
       10., 11.,  6.,  6.,  6.,  8.,  6.,  8., 11.,  8.,  9., 12.,  9.,
        8.,  6., 12.,  9.,  9.,  6.,  5.,  6., 12., 10.,  4.,  5.,  5.,
        9.,  6., 10., 12.,  5.,  6.,  4.,  4.,  5.,  9.,  6.,  6., 11.,
        6.,  7.,  4.,  5.,  6., 10.,  8.,  5.,  4.,  5.,  8.,  8., 11.,
        5., 11.,  9., 11., 11.,  6.,  8.,  6., 10.,  9.,  4., 12.,  7.,
        6.,  6., 12., 10.,  4.,  6., 10.,  8.,  4.,  6.,  6.,  4.,  6.,
       11., 12.,  5., 11., 12.,  5.,  8.,  8., 10.,  7.,  7.,  5., 13.,
       12.,  5.,  4., 11.,  7., 13.,  5., 12.,  6.,  4.,  5.,  5.,  4.,
       11.,  9.,  6., 10., 12.,  4.,  4., 11.,  5.,  4.,  9.,  4.,  9.,
        6., 11.,  8.,  3.,  4.,  6.,  5.,  4.,  8.,  9., 10.,  6.,  7.,
        5., 11.,  9.,  4.,  5.,  4.,  3.,  6.,  5.,  5.,  3.,  7.,  4.,
        9.,  8.,  6.,  3.])

fr_1 = torch.tensor([ 9.,  8.,  9., 10., 11.,  6.,  7.,  9., 10., 11.,  5.,  6.,  4.,
        8.,  9.,  5.,  8.,  5.,  9.,  7.,  7.,  7.,  6., 10., 12., 10.,
       11.,  6.,  5.,  8.,  9., 10., 10.,  9., 11.,  5.,  7.,  7.,  6.,
        7.,  8.,  8.,  7.,  7.,  6.,  7.,  8.,  8.,  6.,  5., 13., 10.,
        4.,  5.,  5.,  9., 10., 10., 12.,  5.,  6., 10., 11., 11., 10.,
        7.,  6., 13.,  6., 11.,  8.,  7., 11., 14.,  6., 11., 10., 10.,
        6.,  5., 12.,  7., 11., 12.,  9.,  9.,  9.,  9.,  8.,  7., 11.,
       13.,  4.,  5., 10.,  9., 11.,  5.,  5.,  3.,  6.,  7.,  5., 10.,
        8.,  8., 13., 11., 14., 12., 14.,  4., 11.,  4., 11., 12.,  9.,
        6.,  5.,  8.,  7.,  6., 10.,  9.,  8., 11.,  5.,  6.,  7., 10.,
        7., 11.,  5.,  9.,  8., 10., 12.,  9.,  8.,  7.,  9.,  6.,  5.,
        5.,  9.,  6., 10.,  4., 12.,  9., 12.,  7.,  4.,  9.,  8., 12.,
        6., 12., 10.,  2.,  6., 12.,  7.,  8.,  7.,  7.,  8.,  8., 12.,
        7., 11., 10.,  7.,  9.,  8.,  8.,  5., 10., 10.,  8.,  6.,  4.,
        3.,  8., 10.,  4.,  4., 10., 11.,  6., 11., 12.,  9.,  5.,  8.,
        7.,  5.,  5.,  9.,  8.,  7.,  9.,  8., 10.,  6.,  6., 10.,  6.,
        5.,  8., 10., 10.,  5.,  8., 10., 10.,  3., 10.,  4.,  5.,  6.,
       13.,  7.,  4.,  9.,  7.,  7.,  8.,  3., 10., 12.,  6., 10., 11.,
        8.,  9.,  5.,  6.,  4.,  7.,  6.,  6.,  6., 10.,  5., 10., 11.,
        7.,  8.,  5.,  4.,  7.,  3.,  7.,  7.,  5.,  9., 11.,  6.,  5.,
       11.,  8.,  6.,  5.,  5.,  9.,  9., 12.,  3.,  6.,  9.,  4.,  6.,
        6., 11.,  4.,  9.,  9.,  8., 10., 12.,  7., 13.,  6.,  8.,  4.,
       12.,  8.,  9.,  4.,  7.,  9.,  7., 10.,  9.,  6.,  9.,  6., 10.,
        9.,  6.,  3.,  8.,  9.,  6., 13.,  9., 10.,  8.,  7.,  5.,  5.,
        6.,  5.,  5.,  6., 12.,  7.,  6.,  9.,  6., 11.,  7.,  7., 11.,
        4., 10.,  3.,  7., 10.,  7., 10., 10.,  7., 10.,  7.,  9., 10.,
       13., 10.,  6.,  4., 10.,  3.,  9., 10.,  9.,  6.,  6.,  8., 10.,
       10.,  5.,  8.,  9.,  9., 11.,  7.,  3.,  8., 11.,  8., 10., 10.,
        8., 11.,  5.,  6.,  4., 10.,  4.,  7., 10.,  6.,  7.,  7., 10.,
        5.,  6.,  8.,  8.,  4.,  8.,  4., 13.,  3., 12.,  4., 10., 10.,
        6., 11.,  8., 10.,  5.,  8., 12.,  9.,  7., 10.,  9., 10.,  7.,
        4.,  6.,  9.,  6.,  4.,  9., 11.,  8., 11.,  5.,  5.,  8., 12.,
       11.,  4.,  5.,  8., 10.,  7., 12.,  8.,  6.,  7.,  6.,  4.,  6.,
        6.,  7.,  9.,  9.,  8.,  6.,  8.,  6.,  4., 15., 11., 11., 11.,
        5., 10.,  4.,  9.,  6., 14.,  7.,  5.,  6.,  8., 10.,  7., 10.,
       10.,  9.,  9.,  5.,  6.,  9.,  7.,  8.,  5.,  5.,  8.,  6.,  8.,
        7., 11.,  7.,  6.,  5., 13., 10.,  5.,  7.,  9.,  8.,  8.,  4.,
        4., 12.,  6., 10., 10.,  8., 12., 10.,  6.,  7.,  5.,  6.,  9.,
       10.,  4.,  5.,  9.,  4.,  4.,  8.,  7.,  8.,  6.,  5.,  8.,  7.,
        7.,  9.,  7.,  7.,  4.,  5.,  9., 10.,  8.,  9., 10.,  7., 12.,
        4., 14.,  4., 13.,  6.,  7., 11., 12.,  7.,  6.,  9.,  5., 11.,
        9.,  4.,  9.,  7.,  9.,  4.,  8.,  9., 11.,  4.,  8.,  6.,  6.,
       11.,  9., 10., 11.,  4., 10.,  9.,  7.,  7.,  6., 11.,  5.,  7.,
        6., 10., 13., 12., 10., 11.,  5., 10., 10.,  6.,  8., 11.,  5.,
       11., 10., 11.,  8.,  4.,  5.,  9., 10., 10.,  8.,  3., 10., 10.,
       11., 10.,  6., 11.,  9.,  9., 10., 10.,  6.,  7.,  9.,  8.,  3.,
        9.,  8.,  8.,  4., 10.,  9., 11., 10.,  8.,  8., 10.,  7., 10.,
        5.,  9., 11.,  6.,  7., 10., 11.,  6., 10., 10.,  4., 11.,  6.,
        6.,  5.,  9., 11., 10.,  9., 10.,  9.,  6.,  6.,  9.,  9., 12.,
        7., 10.,  8.,  9.,  9.,  4., 10.,  9., 13.,  6.,  6.,  9., 11.,
       11.,  5.,  8., 11., 15.,  9.,  4.,  9., 12.,  7.,  7.,  5., 13.,
       11.,  9., 12.,  7.,  9., 10.,  7.,  9.,  5.,  9.,  5.,  4., 15.,
        8.,  7., 11., 11., 11.,  5.,  5., 10.,  5.,  4.,  8.,  5.,  4.,
        9.,  5.,  8.,  5.,  5.,  5.,  5., 11.,  4.,  8.,  4., 10.,  7.,
        6., 10.,  5.,  8., 11.,  7.,  7.,  6.,  4., 11.,  9., 12.,  6.,
       11.,  8., 11.,  8.,  5.,  6.,  8.,  4., 10.,  8.,  7.,  5.,  9.,
        5.,  8.,  8.,  4., 11.,  9.,  8.,  6., 10.,  7.,  6.,  7.,  4.,
        6.,  8., 11.,  9., 11.,  9.,  8.,  6.,  9., 12.,  6.,  5., 12.,
        5.,  7., 13.,  8.,  7.,  7.,  9., 11.,  6.,  8., 10.,  9., 12.,
        7.,  7., 10., 10., 10.,  6.,  6., 12.,  8.,  7.,  7.,  8.,  8.,
        6., 10.,  8., 10.,  8.,  6.,  7.,  8.,  8.,  6.,  8.,  8., 11.,
        7.,  7., 12.,  5., 10., 12.,  5.,  4.,  7., 15.,  7., 12., 11.,
        9.,  5.,  5.,  9.,  5.,  4.,  6., 11.,  9.,  5.,  4.,  4.,  6.,
       11.,  7.,  6.,  6.,  8.,  6.,  9.,  9.,  9.,  5.,  9.,  6., 12.,
        9.,  6.,  6.,  8., 12.,  8.,  9.,  4.,  7.,  8., 10.,  5., 12.,
        9., 11., 10.,  8.,  6., 11., 10.,  7.,  6., 13.,  6., 12., 11.,
        8., 14.,  8., 11.,  6.,  7.,  5.,  7.,  8.,  7.,  7.,  8.,  7.,
        5.,  4., 10., 12., 12.,  9., 12.,  9.,  8., 11., 11.,  8.,  5.,
        8.,  5.,  9., 10.,  9., 10.,  4., 10., 10.,  6., 11.,  7.,  5.,
        5.,  8., 10.,  7.,  6.,  9.,  7., 11.,  5.,  6., 14.,  4.,  9.,
        6., 12.,  8., 11.,  6.,  9.,  4.,  8.,  7.,  5.,  7.,  6.,  8.,
        8., 14.,  8., 13.,  8., 13.,  8.,  5., 10.,  4., 10.,  8.,  6.,
       10.,  2.,  8.,  6., 13.,  9.,  6.,  9., 11.,  7., 10.,  6.,  8.,
       10.,  4.,  5.,  4., 11., 13.,  4.,  5.,  9., 10.,  4., 11.,  4.,
        9.,  4., 11.,  6.,  8.,  6.,  8., 13., 11.,  9.,  9.,  8., 11.,
       11., 10.,  7.,  8.,  6., 10., 12.,  7.,  4.,  9., 10., 12.,  7.,
        7.,  6.,  7., 13.,  5.,  8.,  7.,  5.,  6., 12., 11.,  6.,  8.,
        9.,  8., 10.,  9., 11.,  9.,  4.,  8., 10.,  8.,  7.,  4.,  5.,
       10.,  9.,  7., 10., 11., 10.,  6., 11.,  7.,  8.,  8.,  8.,  6.,
        9.,  9.,  6.,  8.,  4.,  5., 11.,  7., 11.,  9.,  8., 11.,  9.,
        5., 11., 14.,  5.,  5.,  9., 11., 13.,  6.,  5., 10.,  5.,  7.,
        6., 12.,  6.,  9., 10.,  8.,  8.,  6.,  5.,  8., 12.,  6.,  6.,
       11.,  9.,  9., 11.,  7.,  7.,  8.,  5.,  8.,  4.,  9.,  7., 11.,
        9., 11.,  9., 11., 12.,  4.,  8., 10.,  6., 12.,  8., 11.,  8.,
       10., 12., 12.,  7.,  8.,  8.,  7.,  9.,  7.,  9., 11.,  7.,  6.,
        6.,  6.,  6.,  6.,  8.,  6.,  6.,  9.,  2.,  8.,  6.,  5.,  8.,
        9.,  8., 10.,  8.,  8., 10., 11., 11.,  9.,  9.,  7.,  8.,  5.,
        9.,  6.,  5., 13.,  9.,  8.,  6.,  7.,  9.,  6.,  8., 13.,  8.,
        5.,  5., 10.,  5., 11.,  7., 10.,  8.,  6.,  4., 11.,  5.,  5.,
       10., 12.,  7.,  6.,  5.,  4.,  8., 10.,  8.,  9.,  8.,  8.,  8.,
       14.,  8., 11.,  8., 13.,  7.,  8.,  6., 12.,  7., 12., 11.,  7.,
        7.,  6.,  9.,  9., 11.,  9.,  5.,  4.,  5., 10.,  8., 10., 11.,
       10.,  4., 12.,  4.,  7., 10.,  5.,  6.,  9., 11.,  7., 10.,  8.,
        7.,  9.,  8.,  6., 11.,  7., 11.,  9.,  7.,  6., 10.,  9.,  8.,
        7.,  6.,  8., 11., 11.,  8., 10.,  9.,  6., 12.,  6.,  7., 10.,
       10., 11., 11., 10.,  5.,  5.,  4., 11., 10.,  8., 12., 10., 10.,
        8.,  3.,  6.,  9., 10., 10.,  5.,  7.,  5.,  9.,  7.,  6.,  5.,
        9.,  3.,  3.,  9.,  8.,  7.,  6.,  5.,  5.,  5.,  8.,  5.,  7.,
       11.,  8., 10., 10.,  7.,  7.,  4.,  5., 10.,  6.,  7.,  7.,  6.,
        4.,  4.,  9.,  6., 10.,  4.,  7.,  6.,  4., 11.,  6.,  7.,  7.,
       14., 11.,  8.,  7.,  8.,  5.,  7., 10.,  9.,  7.,  9.,  7., 12.,
       10., 12.,  7.,  6.,  9.,  5., 10., 11., 12.,  9.,  9.,  6., 10.,
        6.,  5., 14., 13., 10., 11.,  6.,  6.,  4.,  8.,  4., 11.,  6.,
       11., 14., 10., 11.,  8.,  9.,  8.,  6.,  7., 11.,  7., 10.,  6.,
        8., 11.,  9., 11., 10.,  7., 10.,  6.,  8., 12.,  6.,  4.,  5.,
        4.,  8.,  4., 10., 10.,  4., 11., 11.,  9., 11.,  5.,  9.,  3.,
       10., 10.,  4.,  9.,  7., 13.,  6., 11., 10.,  7.,  6.,  9., 10.,
        2.,  9.,  6., 12.,  8.,  4., 11.,  7.,  4., 12., 10.,  9.,  9.,
        9.,  4.,  5., 10.,  6.,  9.,  4.,  8.,  4., 12.,  7.,  6., 10.,
       11.,  9., 10.,  9.,  5.,  5., 11.,  8.,  6.,  5.,  6.,  7.,  6.,
        8.,  9.,  9.,  4.,  4.,  5.,  5.,  9., 10.,  6., 10., 10.,  4.,
        8.,  8., 12.,  8.,  4.,  4.,  9., 10.,  9., 10., 11., 10.,  9.,
       11., 13., 12.,  7.,  6., 11.,  8.,  4.,  5.,  6.,  9., 10., 12.,
        8.,  8.,  8., 10.,  4.,  8., 10.,  8.,  6.,  7., 10., 10.,  9.,
        9.,  7.,  7., 10.,  7., 12.,  6.,  6.,  4., 12.,  6.,  7., 13.,
        7.,  8.,  5.,  9., 10.,  6.,  7.,  5.,  4.,  6.,  6.,  9.,  6.,
       10., 11.,  5.,  7.,  6.,  8.,  4.,  8.,  4.,  8., 13.,  7., 11.,
        9., 12.,  5., 11.,  8.,  9.,  6.,  6.,  6.,  7.,  5., 10.,  7.,
       10.,  9., 10.,  7.,  4.,  6.,  6., 12., 10.,  6.,  4., 12.,  5.,
        7.,  7.,  5.,  7.,  5.,  7., 10.,  7.,  9., 10.,  8., 12., 10.,
       10., 13.,  7.,  8., 10.,  8.,  6.,  5., 11.,  4.,  7.,  9.,  6.,
        6.,  6.,  9.,  8., 12.,  9.,  8., 10.,  7.,  6.,  5.,  8.,  6.,
       10., 10., 11., 14.,  5.,  8.,  5.,  5.,  9.,  8., 11.,  9.,  9.,
       10., 12.,  9., 10.,  6., 10.,  4.,  6.,  8.,  6., 10., 10., 11.,
        6.,  9.,  6.,  9.,  6.,  9.,  6.,  8.,  8., 11.,  8.,  7.,  9.,
       10.,  5., 11.,  4.,  7.,  7.,  7., 12.,  5.,  4., 12.,  9., 11.,
        6., 10.,  7.,  5.,  9., 12.,  8.,  5., 12.,  9.,  8.,  5., 10.,
        8.,  9.,  7.,  7.,  7.,  8., 11.,  9.,  5., 10., 11.,  6., 10.,
        6., 10.,  8.,  4.,  6.,  7., 11., 10., 11., 10., 11.,  7., 12.,
        4.,  8.,  5., 11.,  8.,  8.,  8., 11.,  7.,  7.,  7.,  8.,  9.,
        7.,  9., 10., 11.,  9., 10., 12.,  5.,  8., 11.,  4.,  6.,  3.,
       10., 11., 12.,  5., 10.,  8.,  9.,  4.,  8.,  8.,  8.,  6.,  7.,
        7.,  2., 14.,  7.,  6.,  8.,  8.,  8.,  7.,  9., 11.,  4.,  6.,
        9.,  9.,  9., 12.,  5.,  7.,  4.,  8.,  6.,  6.,  5.,  6.,  9.,
        8., 10.,  8., 11.,  8.,  9.,  4.,  9., 10.,  7.,  7.,  4., 10.,
       12.,  5., 11., 11.,  6.,  6.,  9.,  4.,  8.,  9.,  5., 12., 11.,
        9.,  8.,  7.,  5.,  7., 12.,  7., 12.,  5., 11., 12., 10.,  6.,
        4.,  4.,  9.,  9., 10.,  8.,  8.,  8.,  9.,  8., 12.,  8.,  6.,
        7., 10., 11.,  9., 13.,  6.,  5., 12., 11.,  9.,  9.,  8.,  7.,
        7.,  8.,  8.,  4., 10., 12., 10.,  6., 10.,  9.,  7.,  9., 11.,
       10.,  8.,  8.,  7., 10., 10.,  4., 11.,  8.,  9., 11.,  9., 11.,
       11.,  5.,  8.,  7., 10., 12., 11., 11.,  3.,  8.,  7., 13.,  2.,
        4., 10.,  4.,  9.,  7.,  8.,  7.,  7.,  4.,  9.,  8., 10.,  7.,
        6.,  9.,  8.,  7., 14.,  8.,  6., 10., 11.,  7.,  7.,  4.,  8.,
        4.,  7., 11.,  9., 10.,  7., 10.,  4.,  8.,  3.,  5.,  6.,  4.,
       10.,  6.,  9.,  6., 11.,  8.,  9.,  3.,  7.,  8.,  9., 10.,  9.,
        8.,  9.,  8.,  8.,  9.,  7.,  9.,  9., 11.,  4.,  9.,  9.,  7.,
        9., 13.,  8.,  9., 10.,  4.,  6.,  9.,  9.,  9.,  5.,  6.,  6.,
       10., 10.,  4.,  4.,  8.,  9.,  3., 11.,  6.,  7.,  9.,  9.,  5.,
        3., 11.,  9.,  6.,  5.,  6.,  9.,  9., 11.,  6., 11.,  8.,  9.,
        9.,  7.,  8.,  7.,  9.,  6., 13., 11.,  7., 13.,  7.,  8.,  9.,
       13.,  9.,  8.,  8., 10., 11., 10.,  6.,  9.,  7., 10.,  9.,  7.,
       10.,  5.,  6.,  8.,  9.,  6.,  8.,  8.,  7.,  9.,  8.,  8.,  6.,
       10.,  8.,  9.,  7.,  8.,  9.,  8.,  2.,  5.,  6.,  5., 12., 12.,
        7.,  4.,  7.,  8., 10.,  4.,  8.,  9.,  9., 10.,  9.,  4.,  8.,
       11.,  4.,  7.,  8.])

targets_acc_test = torch.tensor([0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1,
        1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0,
        0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1,
        0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1,
        1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1,
        0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0,
        0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0,
        1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1,
        1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
        0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1,
        1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1,
        0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1,
        1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0,
        0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1,
        1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0,
        1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0,
        1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1,
        0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1,
        1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1,
        1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0,
        0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0,
        0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1,
        0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1,
        0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1,
        0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0,
        1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0,
        0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0,
        1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0,
        0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0,
        1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0,
        0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1,
        0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0,
        0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1,
        0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0,
        0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0,
        1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1,
        0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0,
        1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1,
        0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0,
        1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0,
        1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0,
        1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0,
        0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0,
        0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
        0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0,
        1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0,
        0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1,
        0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1,
        1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0,
        0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0,
        0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1,
        0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1,
        0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0,
        0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1,
        1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1,
        0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0,
        1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0,
        0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0,
        1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0,
        0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0,
        0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1,
        0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0,
        0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1,
        1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1,
        0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0,
        1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1,
        0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1,
        1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0,
        0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0,
        1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1,
        1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0,
        0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0,
        1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0,
        0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0,
        0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1,
        1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1,
        0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0,
        0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1,
        1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1,
        1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0,
        1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1,
        0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0,
        1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1,
        0, 1, 1])

# generating a polpulation encoded data with size [batch_size, num_classes]
pop_code = torch.stack([fr_0, fr_1], dim=1)
print("pop_code shape:", pop_code.shape)
_, idx = pop_code.max(1)
accuracy = np.mean((targets_acc_test == idx).detach().cpu().numpy())
print("Hardware Accuracy = ", accuracy*100, "%")